In [ ]:
# Importar bibliotecas
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os

# Diretórios do dataset
# Faça upload do arquivo cats_and_dogs.zip no Colab antes
from zipfile import ZipFile
with ZipFile("cats_and_dogs.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

# Geradores de dados
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'dataset', target_size=(224, 224), batch_size=32, class_mode='binary', subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset', target_size=(224, 224), batch_size=32, class_mode='binary', subset='validation'
)

# Transfer Learning com MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento
history = model.fit(train_generator, validation_data=validation_generator, epochs=5)

# Salvar gráficos de acurácia e perda
os.makedirs('images', exist_ok=True)
plt.figure()
plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.legend()
plt.title('Acurácia')
plt.savefig('images/accuracy_plot.png')

plt.figure()
plt.plot(history.history['loss'], label='Treinamento')
plt.plot(history.history['val_loss'], label='Validação')
plt.legend()
plt.title('Perda')
plt.savefig('images/loss_plot.png')
